In [55]:
import requests as rq
import csv
import numpy as np

In [56]:
covid_tracking = rq.get("https://covidtracking.com/api/v1/states/current.json").json()

In [57]:
with open("states_coords.csv", "r") as f:
    csv_file = csv.reader(f, delimiter=",")
    csv_unpacked = [row for row in csv_file]
    states_coords = [{csv_unpacked[0][i]: row[i] for i in range(len(row))} for row in csv_unpacked]

In [58]:
for covid_row in covid_tracking:
    for coords_row in states_coords:
        if covid_row['state']==coords_row['state']:
            covid_row['longitude']=coords_row['longitude']
            covid_row['latitude']=coords_row['latitude']

In [65]:
covid_tracking

[{'state': 'AK',
  'positive': 257,
  'positiveScore': 1,
  'negativeScore': 1,
  'negativeRegularScore': 1,
  'commercialScore': 1,
  'grade': 'A',
  'score': 4,
  'negative': 7475,
  'pending': None,
  'hospitalizedCurrently': None,
  'hospitalizedCumulative': 31,
  'inIcuCurrently': None,
  'inIcuCumulative': None,
  'onVentilatorCurrently': None,
  'onVentilatorCumulative': None,
  'recovered': 63,
  'lastUpdateEt': '4/11 14:45',
  'checkTimeEt': '4/11 16:39',
  'death': 8,
  'hospitalized': 31,
  'total': 7732,
  'totalTestResults': 7732,
  'posNeg': 7732,
  'fips': '02',
  'dateModified': '2020-04-11T18:45:00Z',
  'dateChecked': '2020-04-11T20:39:00Z',
  'notes': 'Please stop using the "total" field. Use "totalTestResults" instead.',
  'hash': '9a40c45c0a168b48c7e6db4484946abc1dee37bb',
  'longitude': '-154.493062',
  'latitude': '63.588753'},
 {'state': 'AL',
  'positive': 3262,
  'positiveScore': 1,
  'negativeScore': 1,
  'negativeRegularScore': 0,
  'commercialScore': 1,
  'g

In [68]:
{**covid_tracking[0], 'hi':'yo'}

{'state': 'AK',
 'positive': 257,
 'positiveScore': 1,
 'negativeScore': 1,
 'negativeRegularScore': 1,
 'commercialScore': 1,
 'grade': 'A',
 'score': 4,
 'negative': 7475,
 'pending': None,
 'hospitalizedCurrently': None,
 'hospitalizedCumulative': 31,
 'inIcuCurrently': None,
 'inIcuCumulative': None,
 'onVentilatorCurrently': None,
 'onVentilatorCumulative': None,
 'recovered': 63,
 'lastUpdateEt': '4/11 14:45',
 'checkTimeEt': '4/11 16:39',
 'death': 8,
 'hospitalized': 31,
 'total': 7732,
 'totalTestResults': 7732,
 'posNeg': 7732,
 'fips': '02',
 'dateModified': '2020-04-11T18:45:00Z',
 'dateChecked': '2020-04-11T20:39:00Z',
 'notes': 'Please stop using the "total" field. Use "totalTestResults" instead.',
 'hash': '9a40c45c0a168b48c7e6db4484946abc1dee37bb',
 'longitude': '-154.493062',
 'latitude': '63.588753',
 'hi': 'yo'}